# Homo NN 自定义Trainer

除了Dataset与CustModel外，还支持对Trainer的自定义，以满足对训练流程的需求, traner的基类位于nn.homo.trainer.trainer_base下，
如果需要开发自己的Trainer，你需要实现一些接口，以让FATE可以正确调用

## TrainerBase接口介绍

## 实例：实现一个能够聚合部分模型的Trainer - TWAFL Trainer

这里参考TWAFL方法(见论文https://arxiv.org/pdf/1903.07424v1.pdf)， 我们实现一个简化版的TWAFL Trainer，它将模型分为浅层与深层，浅层的模型一般来说能够更能捕捉到数据里的有效特征（如卷积层），因此，TWAFL Trainer的特点在于，相比于一整个完整的模型，它会更频繁的聚合浅层模型，当训练达到一定轮数后，再聚合深层模型。这样的设计在联邦学习的过程中，能专注于聚合参数少的浅层，以减少参数的通信量。

本实例仅为演示Trainer的定制化，代码本身未经过任何测试，请勿将其用在实际生产中

### Aggregator

FATE自带SecureAggregatorClient，用于client端的模型聚合，在1.10中可以对Aggregator进行开发定制，在以后教程会提到

使用SecureAggregatorClient，你需要指定最大聚合轮数n（一般为epoch)，并保证调用aggregate接口n次，
aggregate接口接受模型参数以及该epoch的loss作为参数，返回聚合的模型，以及Loss的收敛情况（bool）

代码与接口详见federatedml.framework.homo.aggregator.secure_aggregator

In [7]:
from federatedml.framework.homo.aggregator.secure_aggregator import SecureAggregatorClient

### twafl_trainer.py
下面代码给出了twafl trainer的例子，实现了trainer接口，从代码可见，本地的训练流程与编写一个pytorch的本地训练脚本区别不大，
在实现train/predict接口时，请保证接口的参数与TrainerBase一致，如果predict

**train接口不返回任何东西**
***predict接口如果需要*

In [ ]:
import torch as t
from federatedml.util import LOGGER
from federatedml.nn.homo.trainer.trainer_base import TrainerBase
from torch.utils.data import DataLoader
# 使用FATE自带的SecureAggregator，开发Trainer时，只需要使用SeureAggregator的Client端
from federatedml.framework.homo.aggregator.secure_aggregator import SecureAggregatorClient


class TWAFLTrainer(TrainerBase):
    
    def __init__(self, epochs, batch_size=256, dataloader_worker=4, deep_agg_round=10):
        super(CustTrainer, self).__init__()
        self.epochs = epochs
        self.batch_size = batch_size
        self.dataloader_worker = dataloader_worker
        self.deep_agg_round = deep_agg_round
        
    # 实现train 接口
    def train(self, train_set, val=None, optimizer=None, loss=None):
        
        fed_avg = None
        LOGGER.info('run local mode is {}'.format(self.fed_mode))
        
        # 当调用trainer.local_mode()时，会将fed_mode设定为False，加入此判断是为了满足
        # 本地测试的需要，可以绕过联邦的流程，否则会报错
        if self.fed_mode:
            # max aggregate round 为多聚合轮数
            # sample number用于计算模型权重
            fed_avg = SecureAggregatorClient(max_aggregate_round=self.epochs, sample_number=len(train_set), secure_aggregate=True)
            LOGGER.info('initializing fed avg')
        
        # dataloader + for 循环， 算的loss并backward
        # 与pytorch的训练流程完全一致
        dl = DataLoader(train_set, batch_size=self.batch_size, num_workers=self.dataloader_worker)
        for epoch_idx in range(0, self.epochs):
            l_sum = 0
            for data, label in dl:
                optimizer.zero_grad()
                pred = self.model(data)
                l = loss(pred, label)
                l.backward()
                optimizer.step()
                l_sum += l
                
            LOGGER.info('loss sum is {}'.format(l_sum))
            
            # 通过secure aggregator聚合模型即可
            if fed_avg:
                if (epoch_idx + 1) % self.deep_agg_round == 0:  # 当满足一定轮数，我们聚合完整模型与loss
                    fed_avg.aggregate(self.model, l_sum.cpu().detach().numpy())
                else:
                    # 否则 仅仅聚合浅层模型 这要求模型提供属性shallow_model
                    fed_avg.aggregate(self.model.shallow_model, l_sum.cpu().detach().numpy()) 
                    
        LOGGER.info('training finished!')